In [36]:
import pandas as pd
from plotly.subplots import make_subplots
import plotly.graph_objects as go
subway=pd.read_csv('./DATA/001_1.MONTHLY_SUBWAY_INFO.csv', encoding='utf-8')
covid=pd.read_csv('./DATA/002_2.COVID-19.csv', encoding='utf-8')

###### 전처리

In [37]:
# 월간 전철 이용자
submonth=subway.groupby('date').mean().drop('wd',axis=1).sum(axis=1)
submonth=pd.DataFrame(submonth)
submonth=submonth.reset_index()

submonth['date']=submonth['date'].astype('str')
submonth['date']=pd.to_datetime(submonth['date'],format="%Y%m")
submonth.columns=['month','sub_count']
submonth

,month,sub_count
0,2015-01-01,791602.212341
1,2015-02-01,683333.985455
2,2015-03-01,861907.007220
3,2015-04-01,850038.761733
4,2015-05-01,840768.242806
...,...,...
74,2021-03-01,592109.074257
75,2021-04-01,585458.266447
76,2021-05-01,573910.333882
77,2021-06-01,590010.143092


In [38]:
# 월간 확진자 수
freq = covid.groupby(['DATE']).count()
covid_seoul=freq['ID']
covid_seoul=covid_seoul.rename('covid')
covid_seoul=pd.DataFrame(covid_seoul)
covid_seoul=covid_seoul.reset_index()

# 일간 'DATE' 열을 월간 'month'으로 전처리
covid_seoul['DATE']=pd.to_datetime(covid_seoul['DATE'])
covid_seoul['year']=covid_seoul['DATE'].dt.year
covid_seoul['month']=covid_seoul['DATE'].dt.month

covmonth=covid_seoul.groupby(['year','month']).sum()
covmonth=covmonth.reset_index()
covmonth['month']=(covmonth['year'].astype(str)
                   +covmonth['month'].astype(str).str.zfill(2))
covmonth=covmonth.drop(['year'],axis=1)

covmonth['month']=pd.to_datetime(covmonth['month'],format='%Y%m')
covmonth

,month,covid
0,2020-01-01,7
1,2020-02-01,80
2,2020-03-01,391
3,2020-04-01,156
4,2020-05-01,229
5,2020-06-01,459
6,2020-07-01,281
7,2020-08-01,2415
8,2020-09-01,1306
9,2020-10-01,733


In [39]:
# 월간 전철 이용자 df과 월간 확진자 수 df을 병합
sub_cov_month=pd.merge(submonth,covmonth, how='outer',on='month')

sub_cov_month = sub_cov_month.fillna(0)

sub_cov_month['sub_count']=sub_cov_month['sub_count'].astype('int64')
sub_cov_month['covid']=sub_cov_month['covid'].astype('int64')

# 2019년 1월부터 2021년 7월까지만 잘라내기
sub_cov_f2019=sub_cov_month[48:79]
sub_cov_f2019

,month,sub_count,covid
48,2019-01-01,740378,0
49,2019-02-01,635177,0
50,2019-03-01,780422,0
51,2019-04-01,784890,0
52,2019-05-01,813598,0
53,2019-06-01,741964,0
54,2019-07-01,766513,0
55,2019-08-01,733163,0
56,2019-09-01,715539,0
57,2019-10-01,801417,0


###### 시각화

In [40]:
# 월간 전철 이용자 그래프 그리기
fig = go.Figure()
fig = make_subplots(specs=[[{'secondary_y': True}]])

fig.add_trace(go.Scatter(x=sub_cov_f2019['month'], y= sub_cov_f2019['sub_count'], mode='lines'
                         , name ='전철이용자 수', line=dict(color='#22741C', width=1)), secondary_y=False)  
fig.update_layout(autosize=False,
                  width=1000,
                  height=600,
                  title='월간 전철 이용자',
                  showlegend= True,
                  title_x=0.5,
                # #   축 range 설정(xaxis, yaxis1 yaxis2)
                #   yaxis=dict(
                #              range=[1,500000000]
                #   ),
                  font=dict(
                              family = 'Malgun Gothic',
                              size = 18
                  ),
                  
                
)
fig.show()

In [35]:
# 월간 확진자 그래프 그리기
fig = go.Figure()
fig = make_subplots(specs=[[{'secondary_y': True}]])
fig.add_trace(go.Scatter(x=sub_cov_f2019['month'], y= sub_cov_f2019['covid'], mode='lines'
                         , name ='확진자 수', line=dict(color='#F361DC', width=1)), secondary_y=False) 
fig.update_layout(autosize=False,
                  width=1000,
                  height=600,
                  title='월간 확진자',
                  showlegend= True,
                  title_x=0.5,
                # #   축 range 설정(xaxis, yaxis1 yaxis2)
                #   yaxis=dict(
                #              range=[1,500000000]
                #   ),
                  font=dict(
                              family = 'Malgun Gothic',
                              size = 18
                  ),
                  
                
)

fig.show()